# Evaluating Open-Source Large Language Models for Data Extraction from Unstructured Reports on Mechanical Thrombectomy in Patients with Ischemic Stroke

Welcome to this notebook that will show you how to extract data from Thrombectomy reports using Mistral-7b AI locally on a Single GPU

In this notebook we load and run Mistral-7b with QLoRA which is a 4bit quantization technique with no performance degradation.


In [ ]:
#Install packages
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git

## Step 2 - Define quantization parameters through the BitsandBytesConfig from transformers


* load_in_4bit=True: specify that we want to convert and load the model in 4-bit precision.
* bnb_4bit_use_double_quant=True: Use nested quantization for more memory efficient inference and training.
* bnd_4bit_quant_type="nf4": The 4bit integration comes with 2 different quantization types FP4 and NF4. The NF4 dtype stands for Normal Float 4 and is introduced in the QLoRA paper. By default, the FP4 quantization is used.
* bnb_4bit_compute_dype=torch.bfloat16: The compute dtype is used to change the dtype that will be used during computation. By default, the compute dtype is set to float32 but computation can be set to bf16 for speedups.



In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)



In [ ]:
import json
import pandas as pd
import time
from datetime import timedelta
from IPython.display import display, HTML


## Step 3 - Load the Model with quantization

In [ ]:
model_id = "mistralai/Mistral-7B-Instruct-v0.2"
#model_id ="BioMistral/BioMistral-7B"
#model_id = "mistralai/Mixtral-8x7B-Instruct-v0.1"
#model_id = "HuggingFaceH4/zephyr-7b-beta"
#model_id = "argilla/notus-7b-v1"
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(model_id)

# Step 4 - Define prompts examples and JSON template

In [ ]:
json_template = {
  "NIHSS_score": "",
  "symptom_onset":"",
  "occluded_vessel":"",
  "occlusion_side":"",
  "used_materials":"",
  "medication": "",
  "complications": "",
  "outcome": "",
  "TICI_score": "",
  "Area dose product": "",
  "Fluoroscopy time": "",
  "arrival_time": "",
  "puncture_time": "",
  "first_series_time": "",
  "artery_opening_time": ""
  }

instructions= {
    "NIHSS_score": "Der NIHSS (National Institutes of Health Stroke Scale) Wert des Patienten. Eine Messung, die verwendet wird, um die Beeinträchtigung durch einen Schlaganfall zu quantifizieren. Erwartet wird eine ganze Zahl zwischen 0 und 42.",
    "symptom_onset": "Die Zeit, zu der der Patient zum ersten Mal Symptome bemerkte. Erwartetes Format: 'HH:MM'.",
    "occluded_vessel": "Das verstopfte Blutgefäß. Mögliche Werte umfassen spezifische Arterien wie ACI, M1, M2, Basillaris usw.",
    "occlusion_side": "Die Seite des Körpers, auf der die Okklusion lokalisiert ist. Erwartete Werte: 'links', 'rechts', 'nicht zutreffend', 'unklar'.",
    "used_materials": "Die neurointerventionellen Materialien, die während des Verfahrens verwendet wurden. Dies kann Schleusen, Katheter, Ballons, Führungsdraht, Stentretriever und Stents umfassen. Kontrastmittel und Angioanlage nicht berücksichtigen",
    "medication": "Details zu Medikamenten, die dem Patienten während oder nach dem Verfahren verabreicht wurden, Kontrastmittel nicht berücksichtigen.",
    "complications": "Komplikationen, die während des Verfahrens aufgetreten sind. Dieses Feld sollte den Typ oder die Natur der Komplikation beschreiben, falls zutreffend.",
    "outcome": "Das Ergebnis des Verfahrens, das angibt, ob es erfolgreich, teilweise erfolgreich oder erfolglos war.",
    "TICI_score": "Der TICI (Thrombolysis in Cerebral Infarction) Wert nach dem Verfahren, der das Niveau des wiederhergestellten Blutflusses im verstopften Bereich darstellt.",
    "Area dose product": "Die gesamte während des Verfahrens verwendete Strahlendosis, gemessen in Gy*cm².",
    "Fluoroscopy time": "Die Gesamtzeit, in der die Fluoroskopie-Bildgebungstechnik während des Verfahrens verwendet wurde. Typischerweise gemessen in Minuten oder Sekunden.",
    "arrival_time": "Die Zeit, zu der der Patient für das Verfahren in der Einrichtung ankam. Erwartetes Format: 'HH:MM'.",
    "puncture_time": "Die Zeit, zu der die Punktion gemacht wurde, um auf das verstopfte Gefäß zuzugreifen. Erwartetes Format: 'HH:MM'.",
    "first_series_time": "Die Zeit der ersten Serie von angiografischen Bildern, die während des Verfahrens aufgenommen wurden. Erwartetes Format: 'HH:MM'.",
    "artery_opening_time": "Die Zeit, zu der die Arterie während des Verfahrens erfolgreich geöffnet wurde. Erwartetes Format: 'HH:MM'."
}


# Import Data and load into Dataframe

In [ ]:
import pandas as pd
df= pd.read_csv ('thrombectomy.csv')

# Zero-shot-Prompting

In [ ]:
# Only display output (two columns/ BioMistral)

device = "cuda:0"

# Convert JSON template and expected outputs to string format
json_template_str = json.dumps(json_template, ensure_ascii=False, indent=2)


# Function to generate a prompt for each report
def generate_prompt(report, json_template):
    prompt = (f"Bitte analysieren Sie den folgenden Bericht und extrahieren Sie die Daten, um die JSON-Vorlage auszufüllen."
              f"Die Antwort soll ausschließlich auf Deutsch erfolgen. Befolgen Sie dabei die folgenden Beschreibungen:\n"
              f"{instructions}\n"
              f"Hier ist die JSON-Vorlage zum Ausfüllen:\n"
              f"{json_template_str}\n\n"
              f"Report:\n"
              f"{report_text}")

    return Prompt



# Function to process each report
def process_report(report):
    start_time = time.time()  # Start timing
    prompt = generate_prompt(report, json_template)
    encodeds = tokenizer.apply_chat_template(prompt, tokenize=True, padding=True, return_tensors="pt").to(device)  # Use prompt directly
    generated_ids = model.generate(encodeds, max_new_tokens=1000, do_sample=True)
    decoded = tokenizer.batch_decode(generated_ids)

    #Extracting the JSON string from the response
    json_start_pattern = "Output:"
    json_end_pattern = "}"
    try:
        json_start_index = decoded[0].find(json_start_pattern) + len(json_start_pattern)
        json_end_index = decoded[0].find(json_end_pattern, json_start_index) + 1  # +1 to include the closing brace
        json_str = decoded[0][json_start_index:json_end_index]
        return json_str.strip()  # Removing leading/trailing whitespace
    except ValueError:
        return ""


def display_two_columns(report, extracted_data):
    # Convert the extracted_data to a formatted JSON string
    #json_str = json.dumps(extracted_data, indent=2, ensure_ascii=False)

    # Create HTML content with two columns
    html_content = f"""
    <div style="display: flex; width: 100%; flex-wrap: wrap;">
        <div style="flex: 1; padding: 10px; border-right: 2px solid #ccc; max-width: 50%; word-wrap: break-word; white-space: pre-wrap;">
            <h3>Report:</h3>
            <pre style="white-space: pre-wrap; word-wrap: break-word;">{report}</pre>
        </div>
        <div style="flex: 1; padding: 10px; max-width: 50%; word-wrap: break-word; white-space: pre-wrap;">
            <h3>Extracted Data:</h3>
            <pre style="white-space: pre-wrap; word-wrap: break-word;">{json_str}</pre>
        </div>
    </div>
    """
    display(HTML(html_content))

# Process reports and display in two columns
for index, row in df.iterrows():
    report = row['Report']
    json_str = process_report(report)

    if json_str:
        # Display the report and the extracted JSON data in two columns
        display_two_columns(report, json_str)
